In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, 
                                            decoder_hidden_units = 100)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3)

Net parameters: 207,195



Epoch:    1, Loss: 2.5321
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' ee  ne  e', ' h  ne  e ', '   ee  e e', '  ee  e e ', ' te  e e  ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
tensor([-13.2000, -12.9900, -11.8800,  -9.3600,  -9.3000], device='cuda:0')
sample
['onss recee', 'oprKnApe t', 'lmeeeta ae', 'aeanosi. a', 'oadin  e  ']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['e         ', '        . ', '        . ', '        . ', '  t       '],
 ['        . ', '       .  ', '       .  ', '       .  ', '       .  '],
 ['       .  ', '         .', '         .', '         .', '         .'],
 ['         .', '       t  ', '       t  ', '      .   ', '         t']]
tensor([[-13.2000, -13.2200, -13.1100, -12.8000, -12.6500],
        [-14.3700, -14.5500, -14.4400, -14.1800, -14.0300],
        [-14.5000, -15.0200, -14.9100, -14.6200, -14.4700],
        [-14.7500, -15.17


Epoch:    2, Loss: 1.9270
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aote te', 'ehaoee te ', ' aoee te e', ' ale te ee', ' le  e eet']
greedy_search
[' the the t', 'e the the ', ' an the th', ' ante the ', ' an the th']
tensor([-10.6600, -11.2800, -13.1300, -13.4400, -12.8400], device='cuda:0')
sample
['ly\n\nth iga', 'es anleoue', ', ong, th\n', 'npanletdes', 'or on than']
beam_search
[['the the th', 'the the th', ' the the t', 'the the th', ' the the t'],
 [' the the t', 'e the the ', 'e the the ', ' the the t', 'the the th'],
 ['s the the ', 'the the to', ' the the p', 'e the the ', ' the the p'],
 [' the there', 'the the te', ' the there', ' the there', ' the there'],
 [' the the p', 'e the ther', 'ing the th', ' the the p', ' the thes ']]
tensor([[-10.3800, -10.6900, -10.7600, -10.5200, -10.5000],
        [-10.6000, -10.7600, -10.9900, -10.6600, -10.9900],
        [-10.9300, -11.9700, -11.6400, -10.9900, -11.3800],
        [-11.4300, -12


Epoch:    3, Loss: 1.6412
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aute te', 'ehaute te ', 'eaore  e  ', 'eate te   ', 'ele  e   t']
greedy_search
[' the the t', 'en the pan', 'e the the ', 'e the the ', 'e the the ']
tensor([ -9.5700, -13.7000,  -9.4600, -10.3400,  -9.6600], device='cuda:0')
sample
['i gherof f', 'able ris i', 'eecgot ot ', 'oop trerel', 'ke whes th']
beam_search
[[' the the t', 'ing the th', 'he the the', 'he the the', 'e the the '],
 [' the the p', 'in the the', 'e the the ', 'his the th', 'ing the th'],
 [' the there', 'e the the ', 'ing the th', 'he the thi', ' the the t'],
 ['the the th', 'the the th', 'ing the pa', 'his the pa', 'ed the the'],
 [' the ther ', 'ing the pa', 'e the ther', 'he the tha', ' the the p']]
tensor([[ -9.5700, -10.1100,  -9.4800,  -9.8400,  -9.6100],
        [ -9.8000, -10.2000,  -9.7300, -10.9300, -10.3600],
        [-10.1900, -10.2600, -10.1500, -11.7600, -10.6000],
        [-10.4400, -10.94


Epoch:    4, Loss: 1.4991
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' k auterdy', 'e oute ty ', 'eaute  e  ', 'auterte   ', 'ele  ere t']
greedy_search
[' the pawn ', 'e the pawn', 'e the pawn', 'and the pa', 'e the pant']
tensor([-10.4600, -12.0000, -10.9200, -10.8200, -13.2000], device='cuda:0')
sample
['ngez ly tk', 'no. wok)o ', 'ons trecu5', 'the\nnowine', 'le pame P-']
beam_search
[[' the the p', 'ing the th', 'ing the th', 'on the the', 'ing the th'],
 [' the the t', 'ing the pa', 'hing the t', 'he the the', 'e the the '],
 [' the there', 'ing the po', 'hing the p', 'and the th', 'and the th'],
 [' the ther ', 'ing the ca', 'ing the pa', 'and the pa', 'and the pa'],
 [' the pawn ', 'ing the an', 'hing the a', 'and the po', 'ing the pa']]
tensor([[ -9.7800,  -9.7400,  -9.3900, -10.1800,  -9.5100],
        [ -9.7900, -10.8200, -10.0300, -10.6700,  -9.7100],
        [ -9.9300, -11.0900, -10.1200, -10.8000, -10.1700],
        [-10.1000, -11.3


Epoch:    5, Loss: 1.4124
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' k outerdy', 'e outentyr', 'etuterter ', 'autertere ', 'arer ereat']
greedy_search
[' the pawn ', 'e the pawn', 'e the pawn', 'and the pa', 'and the pa']
tensor([ -9.8400, -10.8500, -10.0600, -10.4200, -11.6500], device='cuda:0')
sample
['le isivenc', 'ntthe fovh', 'he. In map', 'ir thetsil', 'he\nppeayed']
beam_search
[['s the the ', 'e the the ', 'e the the ', 'at of the ', 'ing the th'],
 [' the there', 'ing the th', 'his the th', 'at on the ', 'ing the pa'],
 [' the pawn ', 'ing the pa', 'e the ther', 'at of the\n', 'and the pa'],
 [' the the p', 'and the pa', 'his the pa', 'and the pa', 'ing the po'],
 [' the the t', 'e the ther', 'his the po', 'and the th', 'ing the pr']]
tensor([[ -9.5500,  -9.8200, -10.1500,  -9.6500,  -9.9000],
        [ -9.6700,  -9.8300, -10.5400, -10.6900, -10.5100],
        [ -9.7200, -10.3800, -11.0300, -12.0200, -11.1000],
        [-10.2000, -10.

In [12]:
[net.tensor2text(t) for t in net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)[0]]

[['<START> the pawn the pawn t',
  '<START> the the pawn the th',
  '<START> the the pawn the pa',
  '<START> the pawn the there ',
  '<START> the there the the p'],
 ['<START>ing the there the th',
  '<START>ing the there the pa',
  '<START>ing the there there ',
  '<START>ing the there the ma',
  '<START>ing the there the pr'],
 ['<START> the pawn in the paw',
  '<START> the pawn in there t',
  '<START> the pawn the pawn a',
  '<START> the pawn the pawn t',
  '<START> the pawn the pawn i'],
 ["<START>ack's Black's Black ",
  "<START>ack's Black's Black'",
  "<START>ack's Black's Black,",
  "<START>ack's Black's Blacks",
  "<START>ack's Black's Black:"],
 ['<START>of the the pawn the ',
  '<START>and the pawn the the',
  '<START>and the pawn the paw',
  '<START>and the pawn the pra',
  '<START>and the pawn the pre'],
 ['<START>re the pawn.        ',
  '<START>tore the pawn in the',
  '<START>re the pawn.       C',
  '<START>re the pawn.       B',
  '<START>re the pawn.      Ca'],
 ['<